In [67]:
import os
import time

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

import utils
tf.reset_default_graph()
#1.read data in
DATA_FILE = 'data/birth_life_2010.txt'
data, n_samples = utils.read_birth_life_data(DATA_FILE)
for a,b in data[-2:-1]:
    print([a],[b])

[5.287] [55.585587]


In [ ]:
#2.create placeholders for X(birth_rate) and Y (life_expectancy) 
X, Y = tf.placeholder(tf.float32,[1],name='birth_rate'),tf.placeholder(tf.float32,[1],name='life_expectancy')

In [70]:
#3.create weight and bias
w, b = tf.get_variable(name='weights',shape=[1],dtype = tf.float32),tf.get_variable(name='bias',shape=[1],dtype=tf.float32)

#4.build model to predict Y
Y_predicted = w * X + b

#5.use the square error as the loss function
loss = tf.square(Y-Y_predicted,name='loss')

#6.using gradient descent with learning rate of 0.001 to minimize loss
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.001).minimize(loss)

In [ ]:
start = time.time()

# Create a filewriter to write the model's graph to TensorBoard
writer = tf.summary.FileWriter('./graphs/linear_reg',tf.get_default_graph())

with tf.Session() as sess:
    #step 7:initialize the necessary varivables
    sess.run(tf.variables_initializer([w,b]))
    
    #step 8:train the model for 100 epochs
    for i in range(100):
        total_loss = 0
        for x,y in data:
            _,loss_ = sess.run([optimizer,loss],{X:[x],Y:[y]})
            total_loss  += loss_
        print('Epoch {0}:{1}'.format(i,total_loss/n_samples))
    
    #step 9: output the values of w and b
    w_out,b_out = w.eval(),b.eval()
    print(w_out,b_out)
writer.close()
print('Took: %f seconds' %(time.time() - start))

In [ ]:
tf.shape(1)

In [ ]:
plt.plot(data[:,0],data[:,1],'bo',label = 'Real data')
plt.plot(data[:,0],data[:,0] * w_out + b_out,'r',label='Predicted data')
plt.legend()
plt.show()

In [71]:
def huber_loss(labels,predictions,delta = 14.0):
    residual = tf.abs(labels-predictions)
    def f1():return 0.5*tf.square(residual)
    def f2():return delta*residual-0.5*tf.square(delta)
    return tf.cond(residual<delta,f1,f2)

In [ ]:
tf.reshape([1],[])

In [72]:
h_loss = huber_loss(tf.reshape(Y,[]),tf.reshape(Y_predicted,[]))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(h_loss)

In [ ]:
with tf.Session() as sess:
    sess.run(tf.variables_initializer([w,b]))
    for i in range(100):
        for x,y in data:
            sess.run(optimizer,feed_dict={X:[x],Y:[y]})
    print(w.eval(),b.eval())

tf.data.Dataset.from_tensor_slices((features,labels))
tf.data.Dataset.from_generator(gen,output_types,output_shapes)  
用Dataset就不要再构筑X,Y变量了

In [68]:
dataset = tf.data.Dataset.from_tensor_slices((data[:,0],data[:,1]))

In [ ]:
print(dataset.output_types)

In [ ]:
print(dataset.output_shapes)

tf.data.TextLineDataset(filename)
tf.data.FixedLengthRecordDataset(filename)
tf.data.TFRecordDataset(filename)
create an iterator to iterate through samples in Dataset
## iterates through the dataset exactly once.No need to initialize
iterator = dataset.make_one_shot_iterator()
## iterates through the dataset as many times as we want.Need to initialize with each epoch
iterator = dataset.make_initializable_iterator()

In [ ]:
iterator = dataset.make_one_shot_iterator()
X,Y = iterator.get_next()
with tf.Session() as sess:
    print(sess.run([X,Y]))
    print(sess.run([X,Y]))
    print(sess.run([X,Y]))

In [ ]:
tf.shape(X)

In [ ]:
tf.reshape(X,[1])

In [ ]:
tf.shape(Y)

In [69]:
iterator = dataset.make_initializable_iterator()
X,Y = iterator.get_next()

In [75]:
with tf.Session() as sess:
    for i in range(100):
        sess.run(tf.variables_initializer([w,b]))
        sess.run(iterator.initializer)
        total_loss = 0
        try:
            while True:
                sess.run([optimizer])
        except tf.errors.OutOfRangeError:
            pass

dataset = dataset.shuffle(1000)
dataset = dataset.repeat(100)
dataset = dataset.batch(128)
dataset = dataset.map(lambda x:tf.one_hot(x,10))